In [ ]:
import json
import requests
import logging
from datetime import datetime
from collections import defaultdict
from threading import Thread

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("transaction")
logger.setLevel(logging.INFO)
filename = "./logs/transaction_{}.log".format(datetime.utcnow().strftime('%Y_%m_%d_%H:%M:%S'))
file_handler = logging.FileHandler(filename)
file_handler.setLevel(logging.INFO)
console_handler = logging.StreamHandler()
console_handler.setLevel(logging.INFO)
logger.addHandler(file_handler)
logger.addHandler(console_handler)

In [ ]:
def get_information(url, method, params) -> dict:
    headers = {'Content-Type': 'application/json'}
    data = {"jsonrpc":"2.0", "method": method, "params": params, "id":1}
    try:
        r = requests.post(url, headers=headers, data = json.dumps(data))
    except:
        print("Error: request meets error")
        return None
    if r.status_code != 200:
        print("Error: Return status code %s" % r.status_code)
        return None
    try:
        content = json.loads(r.content)
    except ValueError:
        print("Error: Unable to read JSON reply")
        return None
    if "result" in content:
        return content['result']
    else:
        print("Error: The method does not exist/is not available")
        return None

In [ ]:
def getLatestHeader(shard) -> dict:
    url = endpoint[shard]
    method = 'hmy_latestHeader'
    params = []
    return get_information(url, method, params)

In [ ]:
def getBlockTransactionCountByNumber(shard, number) -> int:
    url = endpoint[shard]
    method = "hmy_getBlockTransactionCountByNumber"
    params = [hex(number)]
    num = get_information(url, method, params)
    if num != None:
        return int(num,16)
    else:
        print(f"Error: can't get the transaction number for block: {number}, shard: {shard}")
        print(num)
        return None

In [ ]:
def getBlockByNumber(shard, number) -> dict:
    url = endpoint[shard]
    method = 'hmyv2_getBlockByNumber'
    params = [number, {"fullTx":True,"inclTx":True,"InclStaking":True}]
    res = get_information(url, method, params)
    if res != None:
        return res
    else:
        print(f"Error: can't get block info for block: {number}, shard: {shard}")
        print(res)
        return None

In [ ]:
endpoint = ['https://api.s0.t.hmny.io/', 'https://api.s1.t.hmny.io/', 'https://api.s2.t.hmny.io/', 'https://api.s3.t.hmny.io/']

In [ ]:
shard = {0:[3219002, getLatestHeader(0)['blockNumber']], 1: [787002,getLatestHeader(1)['blockNumber']], 2: [579001, getLatestHeader(2)['blockNumber']], 3:[3158002, getLatestHeader(3)['blockNumber']]}
thread_dict = defaultdict(dict)
for k,v in shard.items():
    prev, curr = v
    thread_lst = defaultdict(list)
    for i in range(prev, curr):
        thread_lst[i%1000].append(i)
    thread_dict[k] = thread_lst

In [ ]:
if __name__ == "__main__":
    total = 8
    def collect_data(s):
        global total
        for i in thread_dict[2][s]:
            count = getBlockTransactionCountByNumber(2, i)
            if count == None:
                # retry the request once
                print("retry getBlockTransactionCountByNumber")
                count = getBlockTransactionCountByNumber(2, i)
            if count != None:
                total += count
                res = getBlockByNumber(2, i)
                if res == None:
                    # retry the request once
                    res = getBlockByNumber(2, i)
                    print("retry getBlockTByNumber")
                if res != None:
                    time = datetime.fromtimestamp(res['timestamp']).strftime('%Y_%m_%d %H:%M:%S')
                    t = {
                        "timestamp": time,
                        "shard": 2,
                         "block": i,
                        "shard-transactions": total,
                        }
                    if s == 1:
                        logger.info(json.dumps(t))
        logger.info(json.dumps(t))

    threads = []
#     for shard in range(len(endpoint)):
    for x in range(1000):
        threads.append(Thread(target = collect_data, args = [x]))
    for t in threads:
        t.start()
    for t in threads:
        t.join()